In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#### References

- [DataprocPySparkBatchOp reference](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.0/google_cloud_pipeline_components.experimental.dataproc.html)
- [Kubeflow SDK Overview](https://www.kubeflow.org/docs/components/pipelines/sdk/sdk-overview/)
- [Dataproc Serverless in Vertex AI Pipelines tutorial](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_dataproc_serverless_pipeline_components.ipynb)
- [Build a Vertex AI Pipeline](https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline)

This notebook is built to run a Vertex AI User-Managed Notebook using the default Compute Engine Service Account.  
Check the Dataproc Serverless in Vertex AI Pipelines tutorial linked above to learn how to setup a different Service Account.  

#### Permissions

Make sure that the service account used to run the notebook has the following roles:

- roles/aiplatform.serviceAgent
- roles/aiplatform.customCodeServiceAgent
- roles/storage.objectCreator
- roles/storage.objectViewer
- roles/dataproc.editor
- roles/dataproc.worker

#### Step 1:
#### Install the required packages

In [25]:
# Google Cloud notebooks requires dependencies to be installed with '--user'
! pip3 install pyspark
! pip3 install --upgrade google-cloud-pipeline-components kfp --user -q
! pip3 install pip install google-auth==2.13.0
# Install latest JDK
! sudo apt-get update
! sudo apt-get install default-jdk -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 17.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=7eecaf1e78c50a25e4e7f8cfa1820f0b2f2bf08cb9d869cd0c6c052347d3beb7
  Stored in directory: /home/jupyter/.cache/pip/wheels/06/51/98/f7a41aad64c08302d6c26c90650e713c3dfeb5cdec4946db00
Successfully built pyspark
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 4.5 MB/s eta 0:00:0000:01
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.17.1
    Uninstalling google-auth-2.17.1:
      Successfully uninstalled google-auth

In [28]:
! pip3 install --upgrade google-cloud-pipeline-components kfp --user -q


#### Once you've installed the additional packages, you may need to restart the notebook kernel so it can find the packages.

Uncomment & Run this cell if you have installed anything from above commands

In [ ]:
# import os

# if not os.getenv("IS_TESTING"):
#    import IPython
#    app = IPython.Application.instance()
#    app.kernel.do_shutdown(True)

#### Step 2:
#### Set Google Cloud properties

**Overview**  
This notebook shows how to build a Vertex AI Pipeline to run a Dataproc Template   
using the DataprocPySparkBatchOp component.

In [ ]:
# User Configuration
# User Inputs

get_project_id = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = get_project_id[0]
REGION = ""  # example "us-west1"
GCS_STAGING_LOCATION = "gs://<bucket_name>" # example "gs://my_bucket_name"
SUBNET = "" # example "projects/<project-id>/regions/<region-id>/subnetworks/<subnet-name>" 
INPUT_HIVE_DATABASE= ""
INPUT_HIVE_TABLES= "*" # example "table1,table2,table3..." or "*"
OUTPUT_BIGQUERY_DATASET= ""
TEMP_BUCKET= "<bucket_name>"
HIVE_METASTORE= "" # example "thrift://shubu-hive2bqnb-m:9083"

## Change if needed
HIVE_OUTPUT_MODE="overwrite"
MAX_PARALLELISM=10 # Controlls number of parallel Dataproc Serverless Jobs

In [35]:
# User Configuration
# User Inputs

get_project_id = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = get_project_id[0]
REGION ="us-central1"
GCS_STAGING_LOCATION = "gs://test-shubu" # example "gs://my_bucket_name"
SUBNET = "projects/yadavaja-sandbox/regions/us-west1/subnetworks/test-subnet1" 
INPUT_HIVE_DATABASE= "default"
INPUT_HIVE_TABLES= "*" # example "table1,table2,table3..." or "*"
OUTPUT_BIGQUERY_DATASET= "hive2bq"
TEMP_BUCKET= "test-shubu"
HIVE_METASTORE= "thrift://shubu-hive2bqnb-m:9083" # example "thrift://shubu-hive2bqnb-m:9083"

## Change if needed
HIVE_OUTPUT_MODE="overwrite"
MAX_PARALLELISM=10 # Controlls number of parallel Dataproc Serverless Jobs

#### Step 3:
#### Import dependencies

In [2]:
import google.cloud.aiplatform as aiplatform
from kfp import dsl
from kfp.v2 import compiler
from datetime import datetime
from google_cloud_pipeline_components.experimental.dataproc import DataprocPySparkBatchOp
import time
import os
from pyspark.sql import SparkSession
import pandas as pd
from pathlib import Path

#### Step 4:
#### Change working directory to the Dataproc Templates python folder

In [3]:
cur_path = Path(os.getcwd())
WORKING_DIRECTORY = os.path.join(cur_path.parent.parent ,'python')

# If the above code doesn't fetches the correct path please
# provide complete path to python folder in your dataproc 
# template repo which you cloned 

# WORKING_DIRECTORY = "/home/jupyter/dataproc-templates/python/"
print(WORKING_DIRECTORY)

/home/jupyter/dataproc-templates/python


In [4]:
%cd $WORKING_DIRECTORY

/home/jupyter/dataproc-templates/python


#### Step 5:
#### Build Dataproc Templates python package

In [5]:
PACKAGE_EGG_FILE = "dist/dataproc_templates_distribution.egg"
! python ./setup.py bdist_egg --output=$PACKAGE_EGG_FILE

/opt/conda/lib/python3.7/site-packages/setuptools/dist.py:547: UserWarning: Normalizing '0.2.0-beta' to '0.2.0b0'
  warnings.warn(tmpl.format(**locals()))
running bdist_egg
running egg_info
writing google_dataproc_templates.egg-info/PKG-INFO
writing dependency_links to google_dataproc_templates.egg-info/dependency_links.txt
writing requirements to google_dataproc_templates.egg-info/requires.txt
writing top-level names to google_dataproc_templates.egg-info/top_level.txt
reading manifest file 'google_dataproc_templates.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'google_dataproc_templates.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
running install_lib
running build_py
copying dataproc_templ

#### Step 6:
#### Copy package to the GCS bucket

For this, make sure that the service account used to run the notebook has the following roles:
 - roles/storage.objectCreator
 - roles/storage.objectViewer

In [ ]:
!gsutil cp main.py $GCS_STAGING_LOCATION/
!gsutil cp $PACKAGE_EGG_FILE $GCS_STAGING_LOCATION/dist/

In [45]:
# User Configuration
# User Inputs

get_project_id = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = get_project_id[0]
REGION ="us-west1"
GCS_STAGING_LOCATION = "gs://test-shubu" # example "gs://my_bucket_name"
SUBNET = "projects/yadavaja-sandbox/regions/us-west1/subnetworks/test-subnet1" 
INPUT_HIVE_DATABASE= "default"
INPUT_HIVE_TABLES= "*" # example "table1,table2,table3..." or "*"
OUTPUT_BIGQUERY_DATASET= "hive2bq"
TEMP_BUCKET= "test-shubu"
HIVE_METASTORE= "thrift://shubu-hive2bqnb-m:9083" # example "thrift://shubu-hive2bqnb-m:9083"

## Change if needed
HIVE_OUTPUT_MODE="overwrite"
MAX_PARALLELISM=10 # Controlls number of parallel Dataproc Serverless Jobs

#### Step 7:
#### Get Hive Tables 
In case user wants to load all the Hive tables from the database, we need to get the table list using the metastore.

Below cell will fetch all tables from the Hive database by running a Spark SQL query using the provided Hive Metastore.

In [46]:
spark.stop()
print(HIVE_METASTORE)
if INPUT_HIVE_TABLES=="*":
    #os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    #os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
    spark=SparkSession.builder \
          .master("local")\
          .appName("Spark Job to get HIVE table list") \
          .config("hive.metastore.uris",HIVE_METASTORE) \
          .enableHiveSupport() \
          .getOrCreate()  
    TABLE_LIST_DF=spark.sql("show tables in "+INPUT_HIVE_DATABASE)
    TABLE_LIST=TABLE_LIST_DF.select("tableName").rdd.flatMap(lambda x: x).collect()
    print("Table Sets to Migrate: ")
    print(TABLE_LIST)
    spark.stop()
else:
    TABLE_LIST=INPUT_HIVE_TABLES.split(",")
    print("Table Sets to Migrate: ")
    print(TABLE_LIST)

thrift://shubu-hive2bqnb-m:9083


/opt/conda/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)
23/04/28 19:20:42 WARN metastore: Failed to connect to the MetaStore Server...
23/04/28 19:20:43 WARN metastore: Failed to connect to the MetaStore Server...
23/04/28 19:20:44 WARN metastore: Failed to connect to the MetaStore Server...
23/04/28 19:20:45 WARN HiveClientImpl: HiveClient got thrift exception, destroying client and retrying (0 tries remaining)
org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
	at org.apache.hadoop.hive.ql.metadata.Hive.getDatabase(Hive.java:1567)
	at org.apache.hadoop.hive.ql.metadata.Hive.databaseExists(Hive.java:1552)
	at org.apache.spark.sql.hive.client.Shim_v0_12.databaseExists(HiveShim.scala:609)
	at org.apache.spark.sql.hive.client.HiveC

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

In [38]:
#%%bash
!cd ./../../python
os.environ["GCP_PROJECT"]=get_project_id[0]
os.environ["REGION"]=REGION
os.environ["GCS_STAGING_LOCATION"]=GCS_STAGING_LOCATION
os.environ["SUBNET"]=SUBNET
os.environ["HIVE_METASTORE"]=HIVE_METASTORE
os.environ[""]=
!echo $HIVE_METASTORE


/bin/bash: line 0: cd: ./../../python: No such file or directory
thrift://shubu-hive2bqnb-m:9083


In [37]:

# export GCP_PROJECT=get_project_id[0]
# export REGION=REGION
# export GCS_STAGING_LOCATION=GCS_STAGING_LOCATION
# export SUBNET=SUBNET
! ./bin/start.sh \
    --properties=spark.hadoop.hive.metastore.uris="thrift://shubu-hive2bqnb-m:9083" \
    -- --template=HIVEDDLEXTRACTOR \
    --hive.ddl.extractor.input.database="default" \
    --hive.ddl.extractor.output.path="gs://test-shubu/hiveddl/output"

/bin/bash: line 0: cd: ./../../python: No such file or directory
total 68
drwxr-xr-x  2 jupyter jupyter 4096 Apr  9 18:03 bin
-rw-r--r--  1 jupyter jupyter  677 Apr  9 18:03 MANIFEST.in
-rw-r--r--  1 jupyter jupyter 3327 Apr  9 18:03 setup.py
-rw-r--r--  1 jupyter jupyter  668 Apr  9 18:03 setup.cfg
-rw-r--r--  1 jupyter jupyter  241 Apr  9 18:03 release-please-config.json
drwxr-xr-x 15 jupyter jupyter 4096 Apr  9 18:03 test
drwxr-xr-x  4 jupyter jupyter 4096 Apr  9 18:57 build
-rw-r--r--  1 jupyter jupyter 9618 Apr 28 18:15 README.md
-rw-r--r--  1 jupyter jupyter   27 Apr 28 18:15 version.py
-rw-r--r--  1 jupyter jupyter  150 Apr 28 18:15 requirements.txt
-rw-r--r--  1 jupyter jupyter 6145 Apr 28 18:15 main.py
drwxr-xr-x 16 jupyter jupyter 4096 Apr 28 18:15 dataproc_templates
drwxr-xr-x  2 jupyter jupyter 4096 Apr 28 18:54 google_dataproc_templates.egg-info
drwxr-xr-x  2 jupyter jupyter 4096 Apr 28 18:54 dist
GCP_PROJECT=yadavaja-sandbox
REGION=us-central1
GCS_STAGING_LOCATION=gs://te

#### Step 8:

Split Hive Tables list based on MAX_PARALLELISM value provided by the user.

In [ ]:
import copy
COMPLETE_LIST = copy.deepcopy(TABLE_LIST)
PARALLEL_JOBS = len(TABLE_LIST)//MAX_PARALLELISM
JOB_LIST = []
while len(COMPLETE_LIST) > 0:
    SUB_LIST = []
    for i in range(MAX_PARALLELISM):
        if len(COMPLETE_LIST)>0 :
            SUB_LIST.append(COMPLETE_LIST[0])
            COMPLETE_LIST.pop(0)
        else:
            break
    JOB_LIST.append(SUB_LIST)
print("List of tables for execution : ")
print(JOB_LIST)+

#### Step 9:

Set Dataproc Template Properties

In [ ]:
PIPELINE_ROOT = GCS_STAGING_LOCATION + "/pipeline_root/dataproc_pyspark"
MAIN_PYTHON_FILE = GCS_STAGING_LOCATION + "/main.py"
JARS = ["gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar"]
PYTHON_FILE_URIS = [GCS_STAGING_LOCATION + "/dist/dataproc_templates_distribution.egg"]

#### Step 10:
#### Build pipeline and run Dataproc Template on Vertex AI Pipelines to migrate Hive tables to BigQuery

For this, make sure that the service account used to run the notebook has the following roles:
 - roles/dataproc.editor
 - roles/dataproc.worker

In [ ]:
runtime_prop={}
runtime_prop['spark.hadoop.hive.metastore.uris']=HIVE_METASTORE
runtime_prop['mapreduce.fileoutputcommitter.marksuccessfuljobs'] = "false"

def migrate_hive(EXECUTION_LIST):
    EXECUTION_LIST = EXECUTION_LIST
    aiplatform.init(project=PROJECT_ID, staging_bucket=GCS_STAGING_LOCATION)

    @dsl.pipeline(
        name="hive-to-bq-pyspark",
        description="Pipeline to migrate tables from hive to bq",
    )
    def pipeline(
        project_id: str = PROJECT_ID,
        location: str = REGION,
        main_python_file_uri: str = MAIN_PYTHON_FILE,
        python_file_uris: list = PYTHON_FILE_URIS,
        jar_file_uris: list = JARS,
        subnetwork_uri: str = SUBNET
    ):
        for table in EXECUTION_LIST:
            BATCH_ID = "hive2bq-{}-{}".format(table,datetime.now().strftime("%s")).replace('_','-')
            TEMPLATE_SPARK_ARGS = [
                                    "--template=HIVETOBIGQUERY",
                                    "--hive.bigquery.input.database={}".format(INPUT_HIVE_DATABASE),
                                    "--hive.bigquery.input.table={}".format(table),
                                    "--hive.bigquery.output.table={}".format(table),
                                    "--hive.bigquery.output.dataset={}".format(OUTPUT_BIGQUERY_DATASET),
                                    "--hive.bigquery.output.mode={}".format(HIVE_OUTPUT_MODE),
                                    "--hive.bigquery.temp.bucket.name={}".format(TEMP_BUCKET)                                    ]
            _ = DataprocPySparkBatchOp(
                project=project_id,
                location=location,
                batch_id=BATCH_ID,
                main_python_file_uri=main_python_file_uri,
                python_file_uris=python_file_uris,
                jar_file_uris=jar_file_uris,
                subnetwork_uri=subnetwork_uri,
                runtime_config_properties=runtime_prop,
                runtime_config_version="1.1", # issue 665
                args=TEMPLATE_SPARK_ARGS,
            )
            time.sleep(5)

    compiler.Compiler().compile(pipeline_func=pipeline, package_path="pipeline.json")

    pipeline = aiplatform.PipelineJob(
            display_name="pipeline",
            template_path="pipeline.json",
            pipeline_root=PIPELINE_ROOT,
            enable_caching=False,
            )
    pipeline.run()

#### Step 11:

Run Dataproc Batch Template based on Hive Tables list calculated in Step 8.

The below cell will call function migrate_hive to migrate tables using dataproc serverless batch job and also add an entry in Audit Table for each Table Set.

In [ ]:
AUDIT_DICT={}
AUDIT_DF = pd.DataFrame(columns=["Source_DB_Name","Source_Table_Set","Target_DB_Name","Target_Table_Set","Job_Start_Time","Job_End_Time","Job_Status"])
 
for execution_list in JOB_LIST:
    print("\n\nLoading Table Set: "+str(execution_list))
    AUDIT_DICT["Source_DB_Name"]=INPUT_HIVE_DATABASE
    AUDIT_DICT["Source_Table_Set"]='|'.join(execution_list)
    AUDIT_DICT["Target_DB_Name"]=OUTPUT_BIGQUERY_DATASET
    AUDIT_DICT["Target_Table_Set"]='|'.join(execution_list)
    AUDIT_DICT["Job_Start_Time"]=str(datetime.now())
    try:
        migrate_hive(execution_list)
    except Exception:
        AUDIT_DICT["Job_Status"]="FAIL"
        print("\n\nSome Error Occured while loading Table Set: "+str(execution_list))
    else:
        AUDIT_DICT["Job_Status"]="PASS"
        print("\n\nLoaded Table Set: "+str(execution_list))

    AUDIT_DICT["Job_End_Time"]=str(datetime.now())
    AUDIT_DF=AUDIT_DF.append(AUDIT_DICT, ignore_index = True)
    
if AUDIT_DF.empty:
    print("Audit Dataframe is Empty")
else:
    print(AUDIT_DF)
    AUDIT_DF.to_csv("gs://"+TEMP_BUCKET+"/audit/audit_file_{}.csv".format(str(datetime.now())),index=False,header = False)